In [1]:
#get authorization from google drive
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# Load original record
import glob
file_path = "input/1880/nypl_1880_81_starred_clean.txt"
with open(file_path) as f:
  data = f.readlines()  

In [3]:
n = len(data)
full_entry = []
for i in range(n):
  ls = data[i].split()
  if ls:
    if ls[0][1:] != "***":
      full_entry.append(' '.join(ls))

In [4]:
full_entry

['1 Zoller Ignatz, shoes, 106 W. 24th',
 '2 Zoller John, tailor. 221 Ninth av.',
 '3 Zoller Robert W. birds, 5 N. William',
 '4 Zoller Stephen, molder, h r 504 W. 55th',
 '5 Zollfrai Sarah, nurse, h 934 First av.',
 '6 Zollfroi Abraham, pedlar, h 202 Seventh',
 '7 Zollfrey Moses, cutter, h 208 E. 49th',
 '8 Zollie Louis, h r 10 Dominick',
 '9 Zollikoffer Oscar, pres. 1545) B’way, h 210 W. 46th',
 '10 Zollikoffer Oscar F. sec. 1549 B’way, h 65 W. 54th',
 '11 Zollinger Caroline, wid. William, h 241 W. 53d',
 '12 Zollmann Leopold, shoes, 344 E. Houston',
 '13 Zollner Charles, express, h 315 W. 35th',
 '14 Zollver Elizabeth, dressmkr. 142 Third',
 '15 Zollver John, tailor, h 142 Third',
 '16 Zolnowski Vincent, physician, h 33 W. 16th',
 '17 Zolvert Auguste, tailor, h 41 Marion',
 '18 Zoncada Frederic W. tinner, h 309 Pearl',
 '19 Zong Joseph, driver, h 203 W. 30th',
 '20 Zonino Michael, laborer, h 203 S. 5th av.',
 '21 Zons Christian T. hats, 121 Third av.',
 '22 Zonzy John, candy, 87 Oliv

In [5]:
import pickle
# Retrieve result
with open("input/1880/prediction.pkl", 'rb') as f:
  d = pickle.load(f)

### Extract different components from prediction results

Only extract information that we need and ignore delimiters.

In [6]:
output = []
for record in d:
  d_record = {}
  d_record["index"] = int(record[0])
  n = len(record[1])
  start = 1
  for i in range(1, n + 1):
    if i <  n:
      #skip delimiters
      if record[2][i] == "D":
        start = i+1
        continue
      elif record[2][i+1] == "D":
        try:
          d_record[record[2][i]].append(record[1][(start-1):i])
        except:
          d_record[record[2][i]] = [record[1][(start-1):i]]
        start = i + 2
    else:
      try:
          d_record[record[2][i]].append(record[1][(start-1):])
      except:
          d_record[record[2][i]] = [record[1][(start-1):]]

  output.append(d_record)

### Further organize prediction results. 

In [7]:
agg = {}
for record in output:
  new = {}

  try:
    occupation = record['OC']
    new['Occupation'] = []
    for oc in occupation:
      new['Occupation'].append(' '.join(oc))
    new['Occupation'] = ' '.join(new['Occupation'])
  except:
    pass

  try:
    # Deal with widow marriage status
    new['Name'] = []
    names = record['NC']
    for name in names:
      name_new = name[0]
      n = len(name)
      for i in range(1,n):
        if name[i] == '.':
          name_new += name[i]
        else:
          name_new += ' '
          name_new += name[i]
      new['Name'].append(name_new)
  
  except:
    pass

  try:
    # Multiple address
    new['Address'] = []
    addresses = record['AC']
    for address in addresses:
      address_new = address[0]
      n = len(address)
      for i in range(1,n):
        if address[i] == '.':
          address_new += address[i]
        else:
          address_new += ' '
          address_new += address[i]
      new['Address'].append(address_new)

  except:
    pass
  
  agg[str(record['index'])] = new

### Generate final fields
Come up with a list of cities.

In [8]:
cities = ['Hoboken', 'N. J.', 'Spain', 'B’klyn', 'Mt. Vernon']
def has_city(address, cities = cities):
  for city in cities:
    if city in address:
      return city
  return ''

For address, if there is a house number then further extract street name and city. If not, assume the address is a city name.

Notice that sometimes house number is not always at the beginning of the address. Example: h r 504 W. 55th

In [9]:
n = len(full_entry)
final_output = {}
# generate fields
for i in range(1,n+1):
  cd = {}
  cd['Full Name'] = ''
  cd['First Name'] = ''
  cd['Last Name'] = ''
  cd['Middle Name'] = ''
  cd['Widow-flag'] = 'False'
  cd['Widow Of'] = ''
  cd['Title'] = ''
  cd['Occupation'] = ''
  cd['Full Address'] = ''
  cd['H Status-flag'] = ''
  cd['H Address'] = ''
  cd['H House Number'] = ''
  cd['H Street Name'] = ''
  cd['H City'] = ''
  cd['Address 2'] = ''
  cd['Address 2 House Number'] = ''
  cd['Address 2 Street Name'] = ''
  cd['Address 2 City'] = ''
  full_entry_ls = full_entry[i-1].split()
  cd['Index'] = full_entry_ls[0]
  cd['Full Entry'] = ' '.join(full_entry_ls[1:])

  record = agg[str(i)]
  # name
  try:
    name = record['Name']
    cd['Full Name'] = name[0]
    name_ls = name[0].split()
    if len(name_ls) == 2:
      cd['Last Name'] = name_ls[0]
      cd['First Name'] = name_ls[1]
    elif len(name_ls) == 3:
      cd['Last Name'] = name_ls[0]
      cd['First Name'] = name_ls[1]
      cd['Middle Name'] = name_ls[2]
    
    #widow flag
    if len(name) == 2:
      name_ls2 = name[1].split()
      if name_ls2[0] == 'widow' or 'wid.':
        cd['Widow-flag'] = 'True'
      if len(name_ls2) > 1:
        cd['Widow Of'] = ' '.join(name_ls2[1:])
  except:
    pass
  
  #occupation
  try:
    cd['Occupation'] = record['Occupation']
  except:
    pass
  
  #address
  try:
    address = record['Address']
    cd['Full Address'] = ','.join(address)
    if len(address) == 1:
      cd['H Address'] = address[0]
      address_1 = address[0].split()
      if address_1[0] == 'h':
        cd['H Status-flag'] = 'True'
        start = 1
      else:
        cd['H Status-flag'] = 'Assume'
        start = 0
      
      house_num = False
      if address_1[start].isdigit():
        house_num = True
      elif address_1[start+1].isdigit():
        house_num = True
        start += 1

      if house_num:
        cd['H House Number'] = address_1[start]
        #city
        city = has_city(cities, address_1)
        if city:
          cd['H City'] = city
          cd['H Street Name'] = ' '.join(address_1[(start+1):]).replace(city,'').strip()
        else:
          cd['H Street Name'] = ' '.join(address_1[(start+1):])
      else:
        cd['H City'] = ' '.join(address_1[start:])

    else:
      add = address[0]
      add_ls = add.split()
      cd['Address 2'] = add
      start = 0
      house_num = False
      if add_ls[start].isdigit():
        house_num = True
      elif add_ls[start+1].isdigit():
        house_num = True
        start += 1
      if house_num:
        cd['Address 2 House Number'] = add_ls[start]
        #city
        city = has_city(add)
        if city:
          cd['Address 2 City'] = city
          cd['Address 2 Street Name'] = ' '.join(add_ls[(start+1):]).replace(city, '').strip()
        else:
          cd['Address 2 Street Name'] = ' '.join(add_ls[(start+1):])
      else:
        cd['Address 2 City'] = ' '.join(add_ls[start:])
      
      add_1 = address[1]
      add_ls = add_1.split()
      if add_ls[0] == 'h':
        cd['H Address'] = add_1
        cd['H Status-flag'] = 'True'
        start = 1
        house_num = False
        if add_ls[start].isdigit():
          house_num = True
        elif add_ls[start+1].isdigit():
          house_num = True
          start += 1
        if house_num:
          cd['H House Number'] = add_ls[start]
          #city
          city = has_city(add_1)
          if city:
            cd['H City'] = city
            cd['H Street Name'] = ' '.join(add_ls[(start+1):]).replace(city, '').strip()
          else:
            cd['H Street Name'] = ' '.join(add_ls[(start+1):])
        else:
          cd['H City'] = ' '.join(add_ls[start:])

  except:
    pass

  final_output[str(i)] = cd

In [10]:
final_output['462']

{'Full Name': 'Zimmermann Marie',
 'First Name': 'Marie',
 'Last Name': 'Zimmermann',
 'Middle Name': '',
 'Widow-flag': 'False',
 'Widow Of': '',
 'Title': '',
 'Occupation': 'strawgds',
 'Full Address': '165 Mercer,h 53 S. Oxford. B’klyn',
 'H Status-flag': 'True',
 'H Address': 'h 53 S. Oxford. B’klyn',
 'H House Number': '53',
 'H Street Name': 'S. Oxford.',
 'H City': 'B’klyn',
 'Address 2': '165 Mercer',
 'Address 2 House Number': '165',
 'Address 2 Street Name': 'Mercer',
 'Address 2 City': '',
 'Index': '462',
 'Full Entry': 'Zimmermann Marie, strawgds. 165 Mercer, h 53 S. Oxford. B’klyn'}

In [11]:
# Save final output
import json
with open('input/1880/final_output.json', 'w') as f:
    json.dump(final_output,f)
f.close()

In [12]:
import pandas as pd
final_output_df = pd.DataFrame(final_output).T

In [13]:
# Save pandas dataframe as pickle object
with open("input/1880/final_output.pkl", 'wb') as f:
    pickle.dump(final_output_df, f)

In [14]:
# Save to csv
final_output_df.to_csv('input/1880/mn_1880_output.csv')